## Tables and Data Source setup

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift endpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
# setTable(tableRt, tableType = "RESOURCE_TABLE")

Redshift endpoint set: 2017-11-13T08:00:07


"beacons_4744"

In [2]:
# Packages
include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")
include("/data/notebook_home/Production/Bodies/Find-Top-WPF-Pages-Body-v1.0.jl")

weeklyTimeVariables(days=1)
#timeVariables(2017,6,8,10,59,2017,6,8,12,59)
#yesterdayTimeVariables()


November 11, 2017 23:59:00 to November 12, 2017 23:59:00 Local Time
November 12, 2017 04:59:00 to November 13, 2017 04:59:00 UTC Time


In [3]:
UP = UrlParamsInit()
UP.beaconTable = table   #table
UP.resourceTable = tableRt
UP.btView = "$(table)_wpf_pview_prod" #localtable
UP.rtView = "$(tableRt)_wpf_pview_prod" #localtableRt
UP.pageGroup = "Your Shot"   #productPageGroup
UP.urlRegEx = "%"   #localUrl
UP.deviceType = "Mobile"

SG = SoastaGraphsInit()
customer = "Nat Geo" 
SG.customer = customer
;

In [4]:
cleanupTableFTWP(tv,UP)
;

page group,page views
Travel WPF,42
Animals WPF,9
Environment WPF,3
Science WPF,1


In [5]:
function wpfLimitedTable(localTable::ASCIIString,table::ASCIIString,productPageGroup::ASCIIString,startTimeMs::Int64, endTimeMs::Int64)
    try
        query("""\
            drop view if exists $localTable
        """)
        
        query("""\
            create or replace view $localTable as 
                (select * from $table where
                    beacon_type = 'page view'
                    and "timestamp" between $startTimeMs and $endTimeMs
                    and page_group = '$(productPageGroup)'
                )
        """)

    catch y
        println("unLimitedTable Exception",y)
    end
end



wpfLimitedTable (generic function with 1 method)

In [6]:
function topUrlTableForWPF(ltName::ASCIIString, pageGroup::ASCIIString,timeString::ASCIIString; rowLimit::Int64=20, beaconsLimit::Int64=2, paginate::Bool=false)
    try

        dbgtopurl = query("""\

        select 
        *
        FROM $(ltName)
        where 
        beacon_type = 'page view'
        limit 10
        """);

        println(nrow(dbgtopurl))
        beautifyDF(dbgtopurl)
        
        topurl = query("""\

        select count(*),
        CASE 
        when  (position('?' in params_u) > 0) then trim('/' from (substring(params_u for position('?' in substring(params_u from 9)) +7)))
        else trim('/' from params_u)
        end urlgroup
        FROM $(ltName)
        where 
        beacon_type = 'page view'
        group by urlgroup
        order by count(*) desc
        limit $(rowLimit)
        """);

        #println(nrow(topurl))
        #beautifyDF(topurl)
        
        if (nrow(topurl) == 0)
            displayTitle(chart_title = "Top $(rowLimit) (min $(beaconsLimit)) URLs for $(pageGroup) - No Page Views", showTimeStamp=false)
            return
        else
            displayTitle(chart_title = "Top $(rowLimit) (min $(beaconsLimit)) URLs for $(pageGroup)", chart_info = ["Note: If you see AEM URL's in this list tell Chris Davis",timeString],showTimeStamp=false)
        end
        
        #scrubUrlToPrint(topurl)
        #println(nrow(topurl))
        
        
        
        newDF = topurl[Bool[x > beaconsLimit for x in topurl[:count]],:]
        printDF = names!(newDF[:,:],[symbol("Views"),symbol("Url - $(pageGroup)")])
        
        #beautifyDF(printDF)

        if (paginate)
            paginatePrintDf(printDF)
        else
            beautifyDF(printDF[:,:])
        end

    catch y
        println("topUrlTable Exception ",y)
    end

end


topUrlTableForWPF (generic function with 1 method)

In [7]:
firstAndLast(UP.btView,"Adventure WPF";rowLimit=10000,beaconsLimit=1,paginate=true);


HTML{UTF8String}("    <table  class=\"dataframe-table\">\n    <thead><tr></tr></thead>\n    <tbody>\n    \n    \n    \n    </tbody>\n")

Top 10000 (min 1) URLs for Adventure WPF - No Page Views

0


In [9]:
firstAndLast(UP.btView,"Animals WPF",rowLimit=10000,beaconsLimit=1,paginate=true);
firstAndLast(UP.btView,"Environment WPF",rowLimit=10000,beaconsLimit=10,paginate=true);
firstAndLast(UP.btView,"Games WPF",rowLimit=10000,beaconsLimit=0,paginate=true);
firstAndLast(UP.btView,"Images WPF",rowLimit=10000,beaconsLimit=0,paginate=true);
firstAndLast(UP.btView,"Movies WPF",rowLimit=10000,beaconsLimit=0,paginate=true);
firstAndLast(UP.btView,"Ocean WPF",rowLimit=10000,beaconsLimit=0,paginate=true);
firstAndLast(UP.btView,"Photography WPF",rowLimit=10000,beaconsLimit=0,paginate=true);
firstAndLast(UP.btView,"Science WPF",rowLimit=10000,beaconsLimit=10,paginate=true);
firstAndLast(UP.btView,"Travel WPF",rowLimit=10000,beaconsLimit=10,paginate=true);

compression types,custom metrics 0,custom metrics 1,custom metrics 2,custom metrics 3,custom metrics 4,custom metrics 5,custom metrics 6,custom metrics 7,custom metrics 8,custom metrics 9,domain,geo cc,geo city,geo lat,geo lon,geo netspeed,geo org,geo postalcode,geo rg,http method,http referrer,http version,key,mobile connection type,page group,params bat lvl,params cmet mpulseid,params cpu cnc,params custom0 st,params custom1 st,params custom2 st,params custom3 st,params custom4 st,params custom5 st,params custom6 st,params custom7 st,params custom8 st,params custom9 st,params dom doms,params dom img,params dom ln,params dom res,params dom res slowest,params dom script,params dom sz,params errors,params h t,params if,params mem total,params mem used,params mob bw,params mob mt,params nt con end,params nt con st,params nt dns end,params nt dns st,params nt domcomp,params nt domcontloaded e,params nt domcontloaded st,params nt domint,params nt domloading,params nt fet st,params nt first paint,params nt load end,params nt load st,params nt nav st,params nt nav type,params nt red cnt,params nt red end,params nt red st,params nt req st,params nt res end,params nt res st,params nt spdy,params nt ssl st,params nt unload end,params nt unload st,params nu,params pgu,params r,params r2,params rt abld,params rt blstart,params rt bmr conen,params rt bmr const,params rt bmr domen,params rt bmr domst,params rt bmr fetst,params rt bmr reqst,params rt bmr resen,params rt bmr resst,params rt bmr secst,params rt bstart,params rt cnf conen,params rt cnf const,params rt cnf domen,params rt cnf domst,params rt cnf fetst,params rt cnf reqst,params rt cnf resen,params rt cnf resst,params rt cnf secst,params rt cstart,params rt end,params rt ntvu,params rt obo,params rt quit,params rt sh,params rt si,params rt sl,params rt srst,params rt start,params rt tstart,params rt tt,params scr bpp,params scr dpx,params scr mtp,params scr orn,params scr xy,params u,params v,params vis lh,params vis lv,params vis st,proxy address,remote ip,session id,session isunload,session latest,session obopages,session pages,session start,session totalloadtime,spdy,ssl,timers before dns,timers boomerang,timers boomr fb,timers boomr lat,timers boomr ld,timers boomr to end,timers custom0,timers custom1,timers custom2,timers custom3,timers custom4,timers custom5,timers custom6,timers custom7,timers custom8,timers custom9,timers dns,timers domload,timers domready,timers fb to boomr,timers loaded,timers missing,timers navst to boomr,timers renderstart,timers ssl,timers t configfb,timers t configjs,timers t domloaded,timers t done,timers t load,timers t page,timers t postrender,timers t prerender,timers t resp,timers tcp,timestamp,url,user agent family,user agent major,user agent minor,user agent mobile,user agent model,user agent os,user agent osversion,user agent raw,warnings,beacon type,site version,ab test,ipv6,user agent manufacturer,user agent device type,user agent isp,geo isp,bandwidth kbps,bandwidth error pc,bandwidth block,headers cf visitor,headers cf ray,headers cf connecting ip,headers x forwarded for,headers x forwarded proto,headers connection,headers host,headers accept encoding,headers accept language,headers accept,headers content length,headers various,params dom img ext,params dom script ext,params ua plt,params ua vnd,http errno,params http method,params http hdr,params http initiator,params h cr,params xhr sync,params rt subres,params bw time,params lat,params lat err,params mob ct,cdim,params nt cinf,params rt ss,errors,tp ga clientid,tp ga utm source,tp ga utm medium,tp ga utm term,tp ga utm content,tp ga utm campaign,tp aa aid,tp aa mid,tp aa campaign,tp ia coreid,tp ia mmc vendor,tp ia mmc category,tp ia mmc placement,tp ia mmc item,tp ia sp type,tp ia sp promotion,tp ia sp link,tp ia re version,tp ia re pagearea,tp ia re link,tp aa purchaseid,beacon id
,1,,,,,,,,,,nationalgeographic.com,GB,Driffield,53.9377,-0.442505,Cable/DSL,TalkTalk,YO25,E1,GET,null,

Top 10000 (min 1) URLs for Animals WPF Note: If you see AEM URL's in this list tell Chris Davis November 11, 2017 23:59:00 to November 12, 2017 23:59:00 Local Time

Views,Url - Animals WPF
7,http://animals.nationalgeographic.com/animals/photos/tigers
2,http://animals.nationalgeographic.com/animals/photos/near-passerines


9


compression types,custom metrics 0,custom metrics 1,custom metrics 2,custom metrics 3,custom metrics 4,custom metrics 5,custom metrics 6,custom metrics 7,custom metrics 8,custom metrics 9,domain,geo cc,geo city,geo lat,geo lon,geo netspeed,geo org,geo postalcode,geo rg,http method,http referrer,http version,key,mobile connection type,page group,params bat lvl,params cmet mpulseid,params cpu cnc,params custom0 st,params custom1 st,params custom2 st,params custom3 st,params custom4 st,params custom5 st,params custom6 st,params custom7 st,params custom8 st,params custom9 st,params dom doms,params dom img,params dom ln,params dom res,params dom res slowest,params dom script,params dom sz,params errors,params h t,params if,params mem total,params mem used,params mob bw,params mob mt,params nt con end,params nt con st,params nt dns end,params nt dns st,params nt domcomp,params nt domcontloaded e,params nt domcontloaded st,params nt domint,params nt domloading,params nt fet st,params nt first paint,params nt load end,params nt load st,params nt nav st,params nt nav type,params nt red cnt,params nt red end,params nt red st,params nt req st,params nt res end,params nt res st,params nt spdy,params nt ssl st,params nt unload end,params nt unload st,params nu,params pgu,params r,params r2,params rt abld,params rt blstart,params rt bmr conen,params rt bmr const,params rt bmr domen,params rt bmr domst,params rt bmr fetst,params rt bmr reqst,params rt bmr resen,params rt bmr resst,params rt bmr secst,params rt bstart,params rt cnf conen,params rt cnf const,params rt cnf domen,params rt cnf domst,params rt cnf fetst,params rt cnf reqst,params rt cnf resen,params rt cnf resst,params rt cnf secst,params rt cstart,params rt end,params rt ntvu,params rt obo,params rt quit,params rt sh,params rt si,params rt sl,params rt srst,params rt start,params rt tstart,params rt tt,params scr bpp,params scr dpx,params scr mtp,params scr orn,params scr xy,params u,params v,params vis lh,params vis lv,params vis st,proxy address,remote ip,session id,session isunload,session latest,session obopages,session pages,session start,session totalloadtime,spdy,ssl,timers before dns,timers boomerang,timers boomr fb,timers boomr lat,timers boomr ld,timers boomr to end,timers custom0,timers custom1,timers custom2,timers custom3,timers custom4,timers custom5,timers custom6,timers custom7,timers custom8,timers custom9,timers dns,timers domload,timers domready,timers fb to boomr,timers loaded,timers missing,timers navst to boomr,timers renderstart,timers ssl,timers t configfb,timers t configjs,timers t domloaded,timers t done,timers t load,timers t page,timers t postrender,timers t prerender,timers t resp,timers tcp,timestamp,url,user agent family,user agent major,user agent minor,user agent mobile,user agent model,user agent os,user agent osversion,user agent raw,warnings,beacon type,site version,ab test,ipv6,user agent manufacturer,user agent device type,user agent isp,geo isp,bandwidth kbps,bandwidth error pc,bandwidth block,headers cf visitor,headers cf ray,headers cf connecting ip,headers x forwarded for,headers x forwarded proto,headers connection,headers host,headers accept encoding,headers accept language,headers accept,headers content length,headers various,params dom img ext,params dom script ext,params ua plt,params ua vnd,http errno,params http method,params http hdr,params http initiator,params h cr,params xhr sync,params rt subres,params bw time,params lat,params lat err,params mob ct,cdim,params nt cinf,params rt ss,errors,tp ga clientid,tp ga utm source,tp ga utm medium,tp ga utm term,tp ga utm content,tp ga utm campaign,tp aa aid,tp aa mid,tp aa campaign,tp ia coreid,tp ia mmc vendor,tp ia mmc category,tp ia mmc placement,tp ia mmc item,tp ia sp type,tp ia sp promotion,tp ia sp link,tp ia re version,tp ia re pagearea,tp ia re link,tp aa purchaseid,beacon id
,1,,,,,,,,,,nationalgeographic.com,IN,Chennai,13.083298,80.283295,Cable/DSL,BSNL,,25,POST,http://envir

...

Top 10000 (min 10) URLs for Science WPF Note: If you see AEM URL's in this list tell Chris Davis November 11, 2017 23:59:00 to November 12, 2017 23:59:00 Local Time

1


HTML{UTF8String}("    <table  class=\"dataframe-table\">\n    <thead><tr><th class=\"cell-1\" style=\"\">compression types</th> <th class=\"cell-2\" style=\"\">custom metrics 0</th> <th class=\"cell-3\" style=\"\">custom metrics 1</th> <th class=\"cell-4\" style=\"\">custom metrics 2</th> <th class=\"cell-5\" style=\"\">custom metrics 3</th> <th class=\"cell-6\" style=\"\">custom metrics 4</th> <th class=\"cell-7\" style=\"\">custom metrics 5</th> <th class=\"cell-8\" style=\"\">custom metrics 6</th> <th class=\"cell-9\" style=\"\">custom metrics 7</th> <th class=\"cell-10\" style=\"\">custom metrics 8</th> <th class=\"cell-11\" style=\"\">custom metrics 9</th> <th class=\"cell-12\" style=\"\">domain</th> <th class=\"cell-13\" style=\"\">geo cc</th> <th class=\"cell-14\" style=\"\">geo city</th> <th class=\"cell-15\" style=\"\">geo lat</th> <th class=\"cell-16\" style=\"\">geo lon</th> <th class=\"cell-17\" style=\"\">geo netspeed</th> <th class=\"cell-18\" style=\"\">geo org</th> <th class=\"cell-19\" style=\"\">geo postalcode</th> <th class=\"cell-20\" style=\"\">geo rg</th> <th class=\"cell-21\" style=\"\">http method</th> <th class=\"cell-22\" style=\"\">http referrer</th> <th class=\"cell-23\" style=\"\">http version</th> <th class=\"cell-24\" style=\"\">key</th> <th class=\"cell-25\" style=\"\">mobile connection type</th> <th class=\"cell-26\" style=\"\">page group</th> <th class=\"cell-27\" style=\"\">params bat lvl</th> <th class=\"cell-28\" style=\"\">params cmet mpulseid</th> <th class=\"cell-29\" style=\"\">params cpu cnc</th> <th class=\"cell-30\" style=\"\">params custom0 st</th> <th class=\"cell-31\" style=\"\">params custom1 st</th> <th class=\"cell-32\" style=\"\">params custom2 st</th> <th class=\"cell-33\" style=\"\">params custom3 st</th> <th class=\"cell-34\" style=\"\">params custom4 st</th> <th class=\"cell-35\" style=\"\">params custom5 st</th> <th class=\"cell-36\" style=\"\">params custom6 st</th> <th class=\"cell-37\" style=\"\">params custom7 st</th> <th class=\"cell-38\" style=\"\">params custom8 st</th> <th class=\"cell-39\" style=\"\">params custom9 st</th> <th class=\"cell-40\" style=\"\">params dom doms</th> <th class=\"cell-41\" style=\"\">params dom img</th> <th class=\"cell-42\" style=\"\">params dom ln</th> <th class=\"cell-43\" style=\"\">params dom res</th> <th class=\"cell-44\" style=\"\">params dom res slowest</th> <th class=\"cell-45\" style=\"\">params dom script</th> <th class=\"cell-46\" style=\"\">params dom sz</th> <th class=\"cell-47\" style=\"\">params errors</th> <th class=\"cell-48\" style=\"\">params h t</th> <th class=\"cell-49\" style=\"\">params if</th> <th class=\"cell-50\" style=\"\">params mem total</th> <th class=\"cell-51\" style=\"\">params mem used</th> <th class=\"cell-52\" style=\"\">params mob bw</th> <th class=\"cell-53\" style=\"\">params mob mt</th> <th class=\"cell-54\" style=\"\">params nt con end</th> <th class=\"cell-55\" style=\"\">params nt con st</th> <th class=\"cell-56\" style=\"\">params nt dns end</th> <th class=\"cell-57\" style=\"\">params nt dns st</th> <th class=\"cell-58\" style=\"\">params nt domcomp</th> <th class=\"cell-59\" style=\"\">params nt domcontloaded e</th> <th class=\"cell-60\" style=\"\">params nt domcontloaded st</th> <th class=\"cell-61\" style=\"\">params nt domint</th> <th class=\"cell-62\" style=\"\">params nt domloading</th> <th class=\"cell-63\" style=\"\">params nt fet st</th> <th class=\"cell-64\" style=\"\">params nt first paint</th> <th class=\"cell-65\" style=\"\">params nt load end</th> <th class=\"cell-66\" style=\"\">params nt load st</th> <th class=\"cell-67\" style=\"\">params nt nav st</th> <th class=\"cell-68\" style=\"\">params nt nav type</th> <th class=\"cell-69\" style=\"\">params nt red cnt</th> <th class=\"cell-70\" style=\"\">params nt red end</th> <th class=\"cell-71\" style=\"\">params nt red st</th> <th class=\"cell-72\" style=\"\">params nt req st</th> <th class=\"cell-73\" style=\"\">params nt res end

Top 10000 (min 10) URLs for Travel WPF Note: If you see AEM URL's in this list tell Chris Davis November 11, 2017 23:59:00 to November 12, 2017 23:59:00 Local Time

10
